In [1]:
import FinanceDataReader as fdr
import cx_Oracle
import pandas as pd

In [2]:
def process_market_data(market_type, stock_code_col, price_col, change_col, change_ratio_col, volume_col, amount_col, marcap_col, include_market=True):
    """
    주어진 시장 데이터를 가져와 STOCKS 테이블에서 STK_ID를 조회한 후, RT_STK 테이블에 저장하는 함수

    :param market_type: str, 'KOSPI', 'KOSDAQ', 'ETF/KR' 중 하나
    :param stock_code_col: str, 종목 코드가 있는 컬럼명 ('Code' 또는 'Symbol')
    :param price_col: str, 가격 컬럼명 ('Close', 'Price')
    :param change_col: str, 변동 금액 컬럼명 ('Changes', 'Change')
    :param change_ratio_col: str, 변동 비율 컬럼명 ('ChagesRatio', 'ChangeRate')
    :param volume_col: str, 거래량 컬럼명 ('Volume')
    :param amount_col: str, 거래대금 컬럼명 ('Amount')
    :param marcap_col: str, 시가총액 컬럼명 ('Marcap', 'MarCap')
    :param include_market: bool, 'Market' 컬럼 포함 여부 (ETF는 False)
    """
    
    # 1️⃣ 시장 데이터 불러오기
    selected_columns = [stock_code_col, 'Name', price_col, change_col, change_ratio_col, volume_col, amount_col, marcap_col]
    if include_market:
        selected_columns.insert(2, 'Market')  # KOSPI, KOSDAQ 데이터에만 Market 포함

    market_list = fdr.StockListing(market_type)[selected_columns]

    # 2️⃣ 결측값(NaN) 처리 및 숫자형 변환 (오류 방지)
    numeric_cols = [price_col, change_col, change_ratio_col, volume_col, amount_col, marcap_col]
    for col in numeric_cols:
        market_list[col] = pd.to_numeric(market_list[col], errors='coerce').fillna(0).astype(float)

    # 3️⃣ 데이터베이스 연결
    dsn = cx_Oracle.makedsn('localhost', 1521, service_name='xe')
    with cx_Oracle.connect(user='c##PROJECT', password='k5002', dsn=dsn) as connection:
        with connection.cursor() as cursor:

            stock_id_mapping = {}

            # 4️⃣ STOCKS 테이블에서 STK_ID 가져오기
            missing_codes = []
            for code in market_list[stock_code_col].unique():
                cursor.execute("SELECT STK_ID FROM STOCKS WHERE STK_CODE = :code", {'code': code})
                result = cursor.fetchone()
                if result:
                    stock_id_mapping[code] = result[0]
                else:
                    missing_codes.append(code)  # STOCKS 테이블에 없는 코드 저장

            # STOCKS에 없는 종목 출력
            if missing_codes:
                print(f"⚠️ STOCKS 테이블에 없는 종목 코드: {missing_codes}")

            # 5️⃣ STK_ID 매핑
            market_list['STK_ID'] = market_list[stock_code_col].map(stock_id_mapping)

            # 6️⃣ STK_ID가 없는 데이터 제외 (오류 방지)
            market_list = market_list.dropna(subset=['STK_ID'])

            # 7️⃣ RT_STK 테이블에 데이터 저장
            for index, row in market_list.iterrows():
                try:
                    cursor.execute("""
                        INSERT INTO RT_STK (RT_STK_ID, STK_ID, PRICE, CHANGE, CHANGE_RATIO, AMOUNT, VOLUME, MARCAP)
                        VALUES (RT_STK_SEQ.NEXTVAL, :stk_id, :price, :change, :change_ratio, :amount, :volume, :marcap)
                    """, {
                        'stk_id': row['STK_ID'],
                        'price': row[price_col],
                        'change': row[change_col],
                        'change_ratio': row[change_ratio_col],
                        'amount': row[amount_col],
                        'volume': row[volume_col], 
                        'marcap': row[marcap_col]
                    })
                except cx_Oracle.DatabaseError as e:
                    error, = e.args
                    print(f"Error occurred while inserting row {index}: {error.message}")

            # 8️⃣ 변경사항 커밋
            connection.commit()

In [3]:
process_market_data('KOSPI', 'Code', 'Close', 'Changes', 'ChagesRatio', 'Volume', 'Amount', 'Marcap', include_market=True)
process_market_data('KOSDAQ', 'Code', 'Close', 'Changes', 'ChagesRatio', 'Volume', 'Amount', 'Marcap', include_market=True)
process_market_data('ETF/KR', 'Symbol', 'Price', 'Change', 'ChangeRate', 'Volume', 'Amount', 'MarCap', include_market=False)